In [1]:
import os

import numpy as np
from archetypal import settings
from pathlib import Path
from uuid import uuid4
import pandas as pd

from utils.constants import EPW_MAP_PATH, EPW_RELATIVE_PATH
from shoeboxer.builder import ShoeboxConfiguration, ShoeBox
from shoeboxer.schedules import schedules_from_seed

# Check if we are running on Windows or Linux using os
if os.name == "nt":
    settings.ep_version == "22.2.0"
    settings.energyplus_location = Path("C:/EnergyPlusV22-2-0")
else:
    settings.ep_version == "22.2.0"
    settings.energyplus_location = Path("/usr/local/EnergyPlus-22-2-0")

c:\tools\Anaconda3\envs\ml-for-bem-final\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package energy-pandas is out of date. Your version is 0.3.3, the latest is 0.3.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [3]:

"""Load Training Source Data"""
data = pd.read_hdf("data/lightning/full_climate_zone/v7/train/monthly.hdf", key="batch_results")
epw_df = pd.read_csv(EPW_MAP_PATH)

"""
Pick Sample to Reproduce
"""
reproduce_ix = 25
features = data.index.to_frame(index=False).iloc[reproduce_ix].to_dict()
original_sim_results = data.iloc[reproduce_ix]

"""
Make Schedules
"""
scheds = schedules_from_seed(
    features["schedules_seed"]
)

"""
Fetch EPW Path
"""
epw_slug = epw_df.iloc[features["base_epw"]]["slug"]
epw_path =EPW_RELATIVE_PATH +"/"+ epw_slug

"""
Copy Features Over
"""
sb_config = ShoeboxConfiguration()
sb_config.width = features["width"]
sb_config.height = features["height"]
sb_config.adiabatic_partition_flag = 0
sb_config.perim_depth = features["perim_depth"]
sb_config.core_depth = features["core_depth"]
sb_config.wwr = features["wwr"]
sb_config.roof_2_footprint = features["roof_2_footprint"]
sb_config.ground_2_footprint = features["ground_2_footprint"]
sb_config.shading_vect = np.array([features[f"shading_{i}"] for i in range(12)])
sb_config.orientation = features["orientation"]
template_dict = features.copy()
template_dict["schedules"] = scheds
sb = ShoeBox(
    name="with-vent",
    epw=epw_path,
    shoebox_config=sb_config,
    template_dict=template_dict,
    output_directory="data/sim_results/"
)

"""
Simulate
"""
idf = sb.idf(run_simulation=False)

hourly_df, monthly_df = sb.simulate(idf=idf)
df = sb.postprocess(monthly_df)
for zone in ["Perimeter", "Core"]:
    for end_use in ["Heating", "Cooling"]:
        val = round(original_sim_results[zone][end_use].sum(),2), round(df[zone][end_use].sum(),2)
        print(zone, end_use, val, round(val[0] - val[1],2))

INFO:shoeboxer.geometry_utils:New perimeter depth: 10.082928097754552
INFO:shoeboxer.geometry_utils:New core depth: 6.3881313542761795
INFO:shoeboxer.geometry_utils:Shifting core origin to perimeter edge; moving 5.082928097754552 m in y axis
INFO:shoeboxer.geometry_utils:Updating adiabatic surfaces.
INFO:shoeboxer.geometry_utils:Maximum shading point is 156.14790584285944
INFO:shoeboxer.geometry_utils:Placing shading around center point [2.3533162 0.       ]
INFO:ShoeBox:Building idf for data\sim_results\with-vent.epjson
INFO:ShoeBox:EnergyPlus Starting
INFO:ShoeBox:EnergyPlus, Version 22.2.0-c249759bad, YMD=2023.11.17 16:38
INFO:ShoeBox:Converted input file format. Exiting.
INFO:ShoeBox:EnergyPlus Run Time=00hr 00min  0.13sec
INFO:ShoeBox:Command executed successfully.


Perimeter Heating (33.09, 33.09) 0.0
Perimeter Cooling (3.27, 3.27) 0.0
Core Heating (14.87, 14.87) 0.0
Core Cooling (11.69, 11.69) 0.0
